In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=8, bias=True)
  (relu): ReLU()
)


In [4]:
train_models(env, models, episodes=20, steps=2000, print_every=2000)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: -0.020000000000010232
------EPISODE 0 rewards------
Model 0: -26.73260564587639
----STEP 0 rewards----
Model 0: 0.9799999999999898
------EPISODE 1 rewards------
Model 0: -29.18725110440718
----STEP 0 rewards----
Model 0: -0.020000000000010232
----LEARNING BEGINS----
------EPISODE 2 rewards------
Model 0: -27.350133105604414
----STEP 0 rewards----
Model 0: 1.9799999999999898
------EPISODE 3 rewards------
Model 0: -28.771647429189755
----STEP 0 rewards----
Model 0: 0.9799999999999898
------EPISODE 4 rewards------
Model 0: -24.470808874150237
----STEP 0 rewards----
Model 0: 1.9799999999999898
[258] [VIRUS] Agent5 ate virus 33
------EPISODE 5 rewards------
Model 0: 145.11371879021863
----STEP 0 rewards----
Model 0: 1.9799999999999898
[304] [VIRUS] Agent6 ate virus 39
------EPISODE 6 rewards------
Model 0: 102.7853527699425
----STEP 0 rewards----
Model 0: -0.020000000000010232
[1355] [VIRUS] Agent7 ate virus 46
------EPISODE 7 rewards------
Model 

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_10_ep20")